## Concrete Crack Detection

In [1]:
import numpy as np
import os

from  PIL import Image
import matplotlib.pyplot as plt
from matplotlib import image
import splitfolders

import tensorflow as tf
import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping

from keras.callbacks import TensorBoard
import time



input_folder='.\\data_concreate\\SDNET2018\\D\\'
output_folder='\\data_concreate\\img\\'


In [2]:
InpShape=64
batch_size = 32 
epochs = 50
TENSORBOARD_LOGS="cement_binary-{}".format(int(time.time())) # log file name for tensorboard

#### Arrange the images to train validation and test sets

In [3]:


# Split with a ratio.
splitfolders.ratio(input_folder, output=output_folder,
        seed=42, ratio=(.7, .2, .1), group_prefix=None, move=False) # default values



Copying files: 13620 files [00:14, 967.66 files/s] 


### Generate Train val and test sets

In [3]:
datagen = ImageDataGenerator(rescale = 1./255.
                                  )
training_set = datagen.flow_from_directory('./'+output_folder + 'train',
                                                 target_size = (InpShape,InpShape),
                                                 batch_size = batch_size,
                                                 shuffle=True,
                                                 class_mode = 'binary')

val_set = datagen.flow_from_directory('./'+output_folder +'val',
                                    
                                            target_size = (InpShape, InpShape),
                                            batch_size = batch_size,
                                            class_mode = 'binary')


test_set = datagen.flow_from_directory('./'+output_folder +'test',
                                            target_size = (InpShape, InpShape),
                                            batch_size = batch_size,
                                            class_mode = 'binary')

Found 28000 images belonging to 2 classes.
Found 8000 images belonging to 2 classes.
Found 4000 images belonging to 2 classes.


### Metrics to Follow

In [4]:
METRICS = [
      keras.metrics.BinaryAccuracy(name='accuracy'),
      keras.metrics.Precision(name='precision'),
      keras.metrics.Recall(name='recall'),
      keras.metrics.TruePositives(name='tp'),
      keras.metrics.FalsePositives(name='fp'),
      keras.metrics.TrueNegatives(name='tn'),
      keras.metrics.FalseNegatives(name='fn'), 
   ]


### MODEL 

In [5]:
inputs = tf.keras.Input(shape=(InpShape,InpShape,3))
x = tf.keras.layers.Conv2D(filters=16, kernel_size=(3, 3), activation='relu')(inputs)
x = tf.keras.layers.MaxPool2D(pool_size=(2, 2))(x)
x = tf.keras.layers.Conv2D(filters=32, kernel_size=(3, 3), activation='relu')(x)
x = tf.keras.layers.MaxPool2D(pool_size=(2, 2))(x)
x = tf.keras.layers.Conv2D(filters=64, kernel_size=(3, 3), activation='relu')(x)
x = tf.keras.layers.MaxPool2D(pool_size=(2, 2))(x)
x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = tf.keras.layers.Dense(16,activation='relu')(x)
outputs = tf.keras.layers.Dense(1, activation='sigmoid')(x)
model = tf.keras.Model(inputs=inputs, outputs=outputs)


In [6]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 64, 64, 3)]       0         
                                                                 
 conv2d (Conv2D)             (None, 62, 62, 16)        448       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 31, 31, 16)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 29, 29, 32)        4640      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 14, 14, 32)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 12, 12, 64)        18496 

In [7]:
model.compile(optimizer=keras.optimizers.Adam(),
              loss=keras.losses.BinaryCrossentropy(), # default from_logits=False
              metrics=METRICS)



early_stop = EarlyStopping(monitor='val_loss', patience=8, verbose=1,mode='auto')
learning_rate = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=3, verbose=1, mode='auto')
tensorboard=TensorBoard(log_dir='./logs/{}'.format(TENSORBOARD_LOGS)) ## logs are under logs folder for tensorboard

### Fit the model

In [8]:
history = model.fit(training_set, validation_data=val_set, 
                     epochs = epochs,callbacks = [learning_rate, early_stop, tensorboard])

Epoch 1/50
875/875 [==============================] - 48s 50ms/step - loss: 0.0843 - accuracy: 0.9702 - precision: 0.9858 - recall: 0.9541 - tp: 13358.0000 - fp: 192.0000 - tn: 13808.0000 - fn: 642.0000 - val_loss: 0.0421 - val_accuracy: 0.9856 - val_precision: 0.9919 - val_recall: 0.9793 - val_tp: 3917.0000 - val_fp: 32.0000 - val_tn: 3968.0000 - val_fn: 83.0000 - lr: 0.0010
Epoch 2/50
875/875 [==============================] - 34s 39ms/step - loss: 0.0437 - accuracy: 0.9855 - precision: 0.9869 - recall: 0.9841 - tp: 13778.0000 - fp: 183.0000 - tn: 13817.0000 - fn: 222.0000 - val_loss: 0.0366 - val_accuracy: 0.9864 - val_precision: 0.9791 - val_recall: 0.9940 - val_tp: 3976.0000 - val_fp: 85.0000 - val_tn: 3915.0000 - val_fn: 24.0000 - lr: 0.0010
Epoch 3/50
875/875 [==============================] - 31s 36ms/step - loss: 0.0378 - accuracy: 0.9875 - precision: 0.9878 - recall: 0.9872 - tp: 13821.0000 - fp: 170.0000 - tn: 13830.0000 - fn: 179.0000 - val_loss: 0.0312 - val_accuracy: 0.98

In [12]:
#history.model.evaluate(test_datagen)
score = model.evaluate(test_set)



125/125 [==============================] - 4s 28ms/step - loss: 0.0151 - accuracy: 0.9955 - precision: 0.9950 - recall: 0.9960 - tp: 1992.0000 - fp: 10.0000 - tn: 1990.0000 - fn: 8.0000


In [10]:
TP=score[4]
FP=score[5]
TN=score[6]
FN=score[7]
ACC=(TP+TN)/(TP+TN+FP+FN)
PRS=TP/(TP+FP)
REC=TP/(TP+FN)
F1=(2*PRS*REC)/(PRS+REC)
print(F1)

TN/(TN+FP)

0.9955022488755622


0.995

In [11]:
model_json = model.to_json()
with open("./model/modelbm.json", "w") as json_file:
    json_file.write(model_json)
model.save_weights("./model/modelbm.h5")

### Test Model

In [74]:
Modelmk_json = "./model/modelbm.json"
Modelmk_weigths = "./model/modelbm.h5"
from tensorflow.python.keras.models import model_from_json
from tensorflow.keras.preprocessing import image

def get_model(modeljson, weights):
    '''
    Function to load saved model and weights 
    '''
    model_json = open(modeljson, 'r')
    loaded_model_json = model_json.read()
    model_json.close()
    model = model_from_json(loaded_model_json)
    model.load_weights(weights)
    return model


def model_predict(img: image, model, dima: int, dimb: int):
    '''
    Get the image data and return predictions
    '''
    img = img.resize((dima, dimb))
    x = tf.keras.preprocessing.image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = x/255
    preds = model.predict(x)

    return preds

In [ ]:

a = '002-30.jpg'  # Image adress to test
im=Image.open(a)
modelmk = get_model(Modelmk_json, Modelmk_weigths)

# Make predictions
predsmk = model_predict(im, modelmk, InpShape, InpShape)[0][0]
pred=predsmk *100
pred=pred.round(1)
pred
